# Описание задания

Сетчатый мир представляет собой некоторое поле размера M x N, в одной из клеток которого находится агент.

Агент может двигаться по сетчатому миру в четырёх направлениях: вверх, вниз, влево и вправо. Однако мир является стохастическим, т.е. с некоторой вероятностью агент может оказаться не в той клетке, где он планировал, а в соседней.

Цель агента – прийти в определенную клетку мира, в которой он получает награду.

![Сетчатый мир](Pic1.png)

В данном задании мы рассмотрим поле размера 10 x 10. Каждая клетка имеет порядковый номер от 1 до 100.
* Возможные действия пронумерованы от 1 до 4.

Агенту удалось собрать опыт взаимодействия с сетчатым миром, который он сохранил в формате записей (𝑠, 𝑎, 𝑟, 𝑠').

Требуется определить какое оптимальное действие агент должен предпринять в каждой клетке мира, чтобы максимизировать свою совокупную награду.
* Коэффициент дисконтирования будущих наград равен 0.95.

# Задание

### Загрузка библиотек

In [1]:
import pandas as pd
from tabulate import tabulate

### Параметры обучения

In [2]:
# Коэффициент дисконтирования будущих наград
gamma = 0.95

# Параметр скорости обучения
learning_rate = 0.5

# Критерий сходимости алгоритма
tol = 1e-3

### Загрузка experience buffer

In [3]:
experience_buffer = pd.read_csv('Grid_world.csv', dtype = int)

In [4]:
experience_buffer.head()

,s,a,r,s'
0,12,3,0,22
1,12,2,0,11
2,12,3,0,11
3,21,3,0,31
4,22,2,0,23


### Определение множеств возможных состояний и допустимых действий

In [5]:
# Множество возможных состояний
state_space = sorted(list(set(experience_buffer['s'])))

# Множество допустимых действий
action_space = sorted(list(set(experience_buffer['a'])))

### Создание Q-матрицы

In [6]:
#####################################################################
#################### Начало оцениваемого задания ####################
#####################################################################

# Создайте Q-матрицу, в которой строки соответствуют возможным состояниям, а столбцы - допустимым действиям
Q_matrix = pd.DataFrame(0.0, index=state_space, columns=action_space)

#####################################################################
#################### Конец оцениваемого задания #####################
#####################################################################

# Копия Q-матрицы на предыдущей итерации для проверки сходимости алгоритма
Q_old = Q_matrix.copy()  

In [7]:
Q_matrix.head()

,1,2,3,4
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0


### Реализация алгоритма Q-обучения

In [8]:
continue_condition = True
i = 0

# Итерационный цикл
while (continue_condition):
    
    # Счетчик количества итераций
    i += 1
    
    # Последовательно обрабатываем каждое наблюдение в experience buffer
    for index, experience in experience_buffer.iterrows():

        # Текущее состояние
        s = experience["s"]
        
        # Выбранное действие
        a = experience["a"]
        
        # Полученная награда
        r = experience["r"]
        
        # Следующее состояние
        s_next = experience["s'"]

        #####################################################################
        #################### Начало оцениваемого задания ####################
        #####################################################################
        
        # Вычислите значение Q-матрицы при совершении действия a в состоянии s. Используйте метод loc.
        Q_s_a = Q_matrix.loc[s, a]
        
        # Вычислите максимальное значение Q-матрицы в состоянии s_next. Используйте методы loc и max.
        Q_next_max = max(Q_matrix.loc[s_next, :])
        
        # Запишите уравнение обновления Q-матрицы. Не забудьте учесть параметр скорости обучения.
        Q_matrix.loc[s,a] = Q_s_a + learning_rate*(r + gamma*Q_next_max - Q_s_a)

        #####################################################################
        #################### Конец оцениваемого задания #####################
        #####################################################################
    
    # Вычисление коэффициента сходимости. После окончания первой итерации он должен быть равен 16009.8365. 
    convergence_rate = (Q_matrix - Q_old).abs().sum().sum()
    print ('Итерация %d завершена. Коэффициент сходимости: %.4f ' % (i, convergence_rate))
    
    # Проверка выполнения условия сходимости
    if (convergence_rate < tol):
        continue_condition = False
    else:
        Q_old = Q_matrix.copy()

# Завершение алгоритма. Алгоритм должен завершиться после 8 итераций.
print ('\nНеобходимый коэффициент сходимости был достигнут.')

Итерация 1 завершена. Коэффициент сходимости: 16009.8365 
Итерация 2 завершена. Коэффициент сходимости: 1582.0720 
Итерация 3 завершена. Коэффициент сходимости: 120.3533 
Итерация 4 завершена. Коэффициент сходимости: 9.2693 
Итерация 5 завершена. Коэффициент сходимости: 0.7137 
Итерация 6 завершена. Коэффициент сходимости: 0.0550 
Итерация 7 завершена. Коэффициент сходимости: 0.0042 
Итерация 8 завершена. Коэффициент сходимости: 0.0003 

Необходимый коэффициент сходимости был достигнут.


### Вывод оптимальной стратегии

In [9]:
# Для каких состояний вывести оптимальные действия?
output_states = [41,42,43,44,45,46,47,48,49,50]

# Вывод оптимальных действий
optimal_strategy = Q_matrix.idxmax(axis=1).loc[output_states].reset_index()
print(tabulate(optimal_strategy.values, headers = ['Состояние', 'Оптимальное действие']))

  Состояние    Оптимальное действие
-----------  ----------------------
         41                       2
         42                       1
         43                       3
         44                       2
         45                       3
         46                       1
         47                       4
         48                       3
         49                       4
         50                       1
